<a href="https://colab.research.google.com/github/damianiRiccardo90/BHP/blob/master/C4-Web_Hackery/Brute_Forcing_Directories_and_File_Locations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *__Brute-Forcing Directories and File Locations__*

The previous example assumed a lot of knowledge about your target. But when you're attacking a custom web application or large e-commerce system, you often won't be aware of all the files accessible on the web server.

Generally, you'll deploy a spider, such as the one included in _Burp Suite_, to crawl the target website in order to discover as much of the web application as possible. But in a lot of cases, you'll want to get ahold of configuration files, leftover development files, debugging scripts, and other security bread-crumbs that can provide sensitive information or expose functionality that the software developer did not intend. The only way to discover this content is to use a brute-forcing tool to hunt down common filenames and directories.

We'll build a simple tool that will accept word lists from common brute forcers, such as the __gobuster__ project (_https://github.com/OJ/gobuster/_) and __SVNDigger__ (_https://www.netsparker.com/blog/web-security/svn-digger-better-lists-for-forced-browsing/_), and attempt to discover directories and files that are reachable on the target web server. You'll find many word lists available on the internet, and you already have quite a few in your Kali distribution (see _/usr/share/wordlists_). For this example, we'll use a list from SVNDigger. You can retrieve the files for SVNDigger as follows:
```
cd ~/Downloads
wget https://www.netsparker.com/s/research/SVNDigger.zip
unzip SVNDigger.zip
```
When you unzip this file, the file __all.txt__ will be in your __Downloads__ directory.

As before, we'll create a pool of threads to aggressively attempt to discover content. Let's start by creating some functionality to create a __Queue__ out of a word-list file. Open up a new file, name it __bruter.py__, and enter the following code:

In [ ]:
import queue
import requests
import threading
import sys

AGENT = "Mozilla/5.0 (X11; Linux x86_64; rv:19.0) Gecko/20100101 Firefox/19."
EXTENSIONS = [".php", ".bak", ".orig", ".inc"]
TARGET = "http://testphp.vulnweb.com"
THREADS = 50
WORDLIST = "/home/tim/Downloads/all.txt"

def get_words(resume=None): #[1]

    def extend_words(word): #[2]
        if '.' in word:
            words.put(f"/{word}")
        else:
            words.put(f"/{word}/") #[3]

        for extension in EXTENSIONS:
            words.put(f"/{word}{extension}")

    with open(WORDLIST) as f:
        raw_words = f.read() #[4]
    
    found_resume = False
    words = queue.Queue()
    for word in raw_words.split():
        if resume is not None: #[5]
            if found_resume:
                extend_words(word)
            elif word == resume:
                found_resume = True
                print(f"Resuming wordlist from: {resume}")
        else:
            print(word)
            extend_words(word)
    return words #[6]
        